In [1]:
import os

In [2]:
print(os.getcwd())

c:\Users\Acer\OneDrive\Desktop\Chest classification Project\End-to-End-Chest-Classification-using-mlflow-DVC\research


### ***Helps to move `back the folder`***

In [3]:
os.chdir('../')

In [4]:
print(os.getcwd()) #This code is got get back the folder.

c:\Users\Acer\OneDrive\Desktop\Chest classification Project\End-to-End-Chest-Classification-using-mlflow-DVC


In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
'''
once it’s defined and returned, you’re only allowed to use the attributes defined in the class, 
and you cannot add or mutate anything else.

'''

@dataclass(frozen=True)
class DataInjectionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
'''

1. Importing all the constants from the `cnnClassifier.constants` module.
2. Importing utility functions `read_yaml` and `create_directories` from the `cnnClassifier.utils.common` module. 

'''

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
'''

The ConfigurationManager class reads configuration and parameter YAML files, prepares necessary directories, 
and returns a structured config object for data ingestion.

'''


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config["artifacts_roots"]])

    def get_data_ingestion_config(self) -> DataInjectionConfig:
        config = self.config["data_ingestion"]  # Make sure this is a dict

        create_directories([config["root_dir"]])  # Create data ingestion folder

        data_ingestion_config = DataInjectionConfig(
            root_dir = config["root_dir"],
            source_url = config["source_url"],
            local_data_file = config["local_data_file"],
            unzip_dir = config["unzip_dir"]
        )

        return data_ingestion_config


In [9]:
import os
import gdown
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size  

class DataIngestion:

    def __init__(self, config: DataInjectionConfig):
        self.config = config

    def download_file(self):
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)

            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir} ......")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info("Download completed successfully.")

        except Exception as e:
            logger.error(f"Download failed: {e}")
            raise e

    def extract_zip_file(self):
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info(f"Extraction completed to {unzip_path}.")

        except Exception as e:
            logger.error(f"Extraction failed: {e}")
            raise e


### ***`Update the pipeline`***

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Data ingestion failed: {e}")

[2025-07-01 22:20:59,787 : INFO: common : YAML file loaded: config\config.yaml]
[2025-07-01 22:20:59,851 : INFO: common : YAML file loaded: params.yaml]
[2025-07-01 22:20:59,855 : INFO: common : Created directory at: artifacts]
[2025-07-01 22:20:59,859 : INFO: common : Created directory at: artifacts/data_ingestion]
[2025-07-01 22:20:59,864 : INFO: 27103143 : Downloading data from https://drive.google.com/file/d/1IC2oaggjc7B6vYAARDWhGhJwYiY5W8yW/view?usp=sharing to artifacts/data_ingestion/data.zip ......]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1IC2oaggjc7B6vYAARDWhGhJwYiY5W8yW
From (redirected): https://drive.google.com/uc?export=download&id=1IC2oaggjc7B6vYAARDWhGhJwYiY5W8yW&confirm=t&uuid=1ae6a74f-c29e-4040-bd2f-f7bd754cef15
To: c:\Users\Acer\OneDrive\Desktop\Chest classification Project\End-to-End-Chest-Classification-using-mlflow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [03:06<00:00, 262kB/s] 

[2025-07-01 22:24:10,321 : INFO: 27103143 : Download completed successfully.]


[2025-07-01 22:24:13,469 : INFO: 27103143 : Extraction completed to artifacts/data_ingestion.]
